Preperation before running the test
1. Register the model/model.joblib in your workspace under the name sklearn-iris
2. Create AKS cluster and attach it to your workspace using AKS creation the commands in ray_aks_setup.yaml
3. Deploy Azure ML web services with following commmand (install azure ml cli v2 if neccessary)
 az online-endpoint create -f endpoint.yml
 az online-deployment create -f blue-deployment.yml
4. Update the environment variables with values in ray_service.yaml line 80-91 with values of a service principal to download model from your workspace (creating one and assign role if neccessary)
5. Run ray service deployment commands in ray_aks_setup.yaml 
6. If you update your code (ray_score.py), then you have to zip the folder script and push to ray repo. Update the line 15 in ray_aks_setup.yaml to point the working_dir to the zip file in your git repo



In [72]:
import json
import urllib.request
import random
import requests

IPADDRESS = '20.252.37.17'
base_uri = f'http://{IPADDRESS}:8000'

def score(X, tenant):
    # tenant = random.choices(["tenant1","tenant2", "tenant3","tenant4","tenant5","tenant6","tenant7", "tenant8","tenant9"])[0]
    len = random.randint(2,50)
    data = {"tenant":tenant, "data": X.head(len).to_numpy().tolist()}

    body = str.encode(json.dumps(data))

    url = f'{base_uri}/score'

    headers = {'Content-Type':'application/json'}

    req = urllib.request.Request(url, body, headers)

    try:
        response = urllib.request.urlopen(req)

        result = response.read()
        print(json.loads(result))
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))

        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))

def updated_dedicated_pool(mapping):

    body = str.encode(json.dumps({"mapping":mapping}))

    url = f'{base_uri}/update_dedicated_pool'

    headers = {'Content-Type':'application/json'}

    req = urllib.request.Request(url, body, headers)

    try:
        response = urllib.request.urlopen(req)

        result = response.read()
        print(json.loads(result))
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))

        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))

def get_dynamic_tenantmap():
    url = f'{base_uri}/get_dynamic_tenantmap'
    headers = {'Content-Type':'application/json'}
    # req = urllib.request.Request(url, headers)

    try:
        # response = urllib.request.Request(url, headers, method="get")
        response = requests.get(url, headers=headers)

        result = response.json()
        print(result)
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))

        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))


In [73]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import joblib
dataset = pd.read_csv("https://azuremlexamples.blob.core.windows.net/datasets/iris.csv")
X= dataset[['sepal_length','sepal_width','petal_length','petal_width']]


In [74]:
score(X, "tenant1")

The request failed with status code: 500
date: Mon, 05 Dec 2022 07:19:11 GMT
server: uvicorn
content-type: text/plain; charset=utf-8
Transfer-Encoding: chunked
Connection: close


Task Error. Traceback: ray::ServeReplica:Dispatcher.handle_request() (pid=322, ip=10.244.0.16)
  File "/home/ray/anaconda3/lib/python3.7/site-packages/ray/serve/_private/utils.py", line 231, in wrap_to_ray_error
    raise exception
  File "/home/ray/anaconda3/lib/python3.7/site-packages/ray/serve/_private/replica.py", line 420, in invoke_single
    result = await method_to_call(*args, **kwargs)
  File "/home/ray/anaconda3/lib/python3.7/site-packages/ray/serve/api.py", line 224, in __call__
    sender,
  File "/home/ray/anaconda3/lib/python3.7/site-packages/fastapi/applications.py", line 269, in __call__
    await super().__call__(scope, receive, send)
  File "/home/ray/anaconda3/lib/python3.7/site-packages/starlette/applications.py", line 124, in __call__
    await self.middleware_stack(scope, receive, send)


In [70]:
get_dynamic_tenantmap()

{"tenant1": "deployment1", "tenant2": "deployment2", "tenant3": "deployment3", "tenant4": "deployment4", "tenant5": "deployment5", "tenant6": "deployment6", "tenant7": "deployment7", "tenant8": "deployment8"}


In [61]:
import json 
import urllib.request
import random
import requests


headers = {'Content-Type':'application/json'}
IPADDRESS = 'localhost'

req = urllib.request.Request(url=f'http://{IPADDRESS}:8000/get_dynamic_tenantmap', headers=headers, method="GET")

with urllib.request.urlopen(req) as res:
  body = json.load(res)
  print(body)

URLError: <urlopen error [Errno 111] Connection refused>

API to update dedicated pool mapping. Try this to assign a tenant to a dedicated deployment

In [53]:
mapping ={"ent_Tenant9":"deployment9","ent_enant10":"deployment10"}
updated_dedicated_pool(mapping)

The request failed with status code: 500
date: Mon, 05 Dec 2022 06:56:13 GMT
server: uvicorn
content-type: text/plain; charset=utf-8
Transfer-Encoding: chunked
Connection: close


Task Error. Traceback: ray::ServeReplica:Dispatcher.handle_request() (pid=12534, ip=172.19.74.241)
  File "/home/hyssh/anaconda3/envs/rayonaml-dev/lib/python3.8/site-packages/ray/serve/_private/utils.py", line 231, in wrap_to_ray_error
    raise exception
  File "/home/hyssh/anaconda3/envs/rayonaml-dev/lib/python3.8/site-packages/ray/serve/_private/replica.py", line 420, in invoke_single
    result = await method_to_call(*args, **kwargs)
  File "/home/hyssh/anaconda3/envs/rayonaml-dev/lib/python3.8/site-packages/ray/serve/api.py", line 221, in __call__
    await self._serve_app(
  File "/home/hyssh/anaconda3/envs/rayonaml-dev/lib/python3.8/site-packages/fastapi/applications.py", line 270, in __call__
    await super().__call__(scope, receive, send)
  File "/home/hyssh/anaconda3/envs/rayonaml-dev/lib/python3.8